In [1]:
import re,requests,math,time
from random import randint
from bs4 import BeautifulSoup
import pandas as pd
import json
from collections import defaultdict

In [2]:
# set type of address

url0 ='https://www.mobile01.com/' #mobile01 homepage
url1 ='waypointtopiclist.php?f=405&p='  
url2 = url0 + 'waypointtopicdetail.php?f=405&'  #mobile01 travel homepage
first_page_url = url0 + url1

In [3]:
first_page_url  # preview first_page_url

'https://www.mobile01.com/waypointtopiclist.php?f=405&p='

In [ ]:
# definite a function for getting all pages link

def get_all_pages_link(first_page_url):
    links = []
    for i in range(1,int(max_page)+1):
        links.append(first_page_url + str(i))
    return links

In [ ]:
# get the max_page of this section
res = requests.get(first_page_url)
soup = BeautifulSoup(res.text,'lxml')
max_page = soup.select('p.numbers')[0].text.split('共')[1].split('頁')[0]  

# definite a function for getting all topic links
def get_all_topic_links(first_page_url):
    links = []
    page = 1
    for url in get_all_pages_link(first_page_url):
        res = requests.get(url)
        soup = BeautifulSoup(res.text,'lxml')
        
        # because article link contain top and gen(stick-top topic and normal topic) 
        # use try and except to avoid error(link is not exist)
        try:
            top_links = []
            for i in [i for i in soup.select('span > a.topic_top')]:
                top_links.append('https://www.mobile01.com/waypointtop' + i['href'].split('top')[1])
            gen_links = []
            for i in [i for i in soup.select('span > a.topic_gen')]:
                gen_links.append('https://www.mobile01.com/waypointtop' + i['href'].split('top')[1])
            print('page{} is OK!'.format(page))
            links += top_links + gen_links
        except:
            gen_links = []
            for i in [i for i in soup.select('span > a.topic_gen')]:
                gen_links.append('https://www.mobile01.com/waypointtop' + i['href'].split('top')[1])
            print('page{} is OK!'.format(page))
            links +=gen_links
        time.sleep(1)
        page+=1
    return links
        

In [ ]:
all_topic_links = get_all_topic_links(first_page_url)  #preview all topic links

In [ ]:
### save all topic links to avoid to continue post request to the server  ###

# for i in all_page_links:
#     f = open("C:\\Users\\Java\\Desktop\\links.text", "a",encoding='utf8')   #workspace
#     f.write(i+'\n')
#     f.close()

In [5]:
### read all topic links from workspace ###

# f = open("C:\\Users\\Java\\Desktop\\links.text", "r",encoding='utf8')  #workspace
# all_link = []
# for i in f:
#     all_link.append(i)

In [8]:
### preview all link ###

# len(all_link)

7702

In [ ]:
# inside_page_link = url0 + soup.select('span > a.topic_gen')[0]['href'] + '&p='

In [ ]:
# url = inside_page_link
# res = requests.get(url)
# soup = BeautifulSoup(res.text,'lxml')
# max_inside_page = int(soup.select('.contentfoot')[0].text.split('共')[1].split('頁')[0])

In [ ]:
### start this function to get all topic and save to the workspace ##

start_topic =1
test = {}
for link in all_topic_links:
    url = link + '&p='
    res = requests.get(url)
    soup = BeautifulSoup(res.text,'lxml')
    max_inside_page = int(soup.select('.contentfoot')[0].text.split('共')[1].split('頁')[0])
    
    author_info = soup.select('div > .single-post-author')[0].text.strip().replace('\n\n','\n').replace('私訊連結','').split('\n')
#     try:
    for s in author_info:
        if s == '無圖示': 
            author_info.remove(s)
    test['作者'] = author_info[0]
    test['標題'] = soup.select('.topic')[0].text 
    test['發文時間'] = author_info[2].split(' ')[0]
    test['人氣'] = author_info[1].split(' ')[1].replace(',','')
    test['來源'] = "mobile01"
    
    test['文章內容'] = soup.select('div.single-post')[0].text.split('\n\n\n\n')[1]
    test['url'] = link
     
    #     

    # #     

    total_author =soup.select('div > .fn')
    
    cm1 =[]
    for i in range(1,len(total_author)):
        author_info = soup.select('div > .single-post-author')[i].text.strip().replace('\n\n','\n').replace('私訊連結','').split('\n')

        
        for s in author_info:
            if s == '無圖示': 
                author_info.remove(s)

        author = author_info[0]
        article_date = author_info[2].split(' ')[0]
        content = soup.select('div.single-post')[i].text.split('\n\n\n\n')[1]
        dd ={'作者':author,'發文日期':article_date,'回文內容':content}
        cm1.append(dd)
    
    time.sleep(randint(3,5))
    

    t2_author = []
    t2_date = []
    t2_content = []
    cm2 = []
    for i in range(2,max_inside_page+1):
        inside_page_url = url + str(i)
        res = requests.get(inside_page_url)
        soup = BeautifulSoup(res.text,'lxml')
        total_author =soup.select('div > .fn')
        
          
        
        for j in range(0,len(total_author)):
            author_info = soup.select('div > .single-post-author')[j].text.strip().replace('\n\n','\n').replace('私訊連結','').split('\n')
            for s in author_info:
                if s == '無圖示': 
                    author_info.remove(s)
            
            author = author_info[0]
            article_date = author_info[2].split(' ')[0]
            content = soup.select('div.single-post')[j].text.split('\n\n\n\n')[1].split('\n\n\n')[0]
#             try:
#                 if re.search('(恕刪)',content):
#                     content = soup.select('div.single-post')[j].text.split('\n\n\n\n')[1].split('(恕刪)')[1]
#                 else:
#                     content = soup.select('div.single-post')[j].text.split('\n\n\n\n')[1]
#             except:
#                 pass
            dd ={'作者':author,'發文日期':article_date,'回文內容':content}
            
            cm2.append(dd)
  
        time.sleep(randint(3,5))
    cm = cm1 + cm2
    
    test['回文'] = cm
    
      

 

    
    dest_path = "C:\\Users\\Java\\Desktop\\mobile01_JP\\command{}.text".format(start_topic)
    with open(dest_path,'a',encoding="utf-8") as f:
#         f.write(str(test))
        f.write(json.dumps(test,sort_keys=True,ensure_ascii=False))
        f.close()
    
    print('第{}則文章以爬取完成'.format(start_topic))
    time.sleep(randint(1,3)) 
    start_topic += 1       

In [ ]:
### use os method to get the file from workspace directory ##
import  os
DATA_DIR = "C:\\Users\\Java\\Desktop\\mobile01_JP\\"
file_data = []
for filename in os.listdir(DATA_DIR):
    print( "Loading: {}".format(filename))
    loadFile = open(os.path.join(DATA_DIR, filename), 'r', encoding='UTF-8')
    file_data.append(loadFile.read())
    loadFile.close()

In [106]:
from pymongo import MongoClient
client = MongoClient('10.120.27.11', 27017)

In [132]:
for i in file_data:
    i = json.loads(i)
    
    client.text_mining.mobile01_article.insert_one(i)